In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from tqdm import tqdm_notebook

In [243]:
links = pd.read_csv('links.csv')
movies = pd.read_csv('movies.csv')
ratings = pd.read_csv('ratings.csv')
tags = pd.read_csv('tags.csv')

In [11]:
% config IPCompleter.greedy = True

### Задание:
Постройте топ фильмов в категориях Action и Comedy

План А:

сформировать топ по 

1) максимальным рейтингам - самые лучшие по качеству и содержанию, 

2) максимальному количеству оценок - самые популярные и просматриваемые, 

3) количеству тегов - самые впечатляющие

План Б: 

все собрать в один обобщающий критерий.

In [4]:
movies.head()

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy


Рейтинг фильма посчитаем медианный

In [244]:
# 1) рейтинг фильма посчитаем медианный
median_rating = ratings.groupby('movieId')[['rating']].median()

In [245]:
# 2) количество оценок
rating_count = ratings.groupby('movieId')[['userId']].count()


In [246]:
# 3) кол-во тегов
tag_count = tags.groupby('movieId')[['tag']].count()

для удобства расчета общего скоринга пронормируем рейтинги и кол-во тегов

In [247]:
# получатся значения в диапазоне от -0,62 до 0.38, где 0 соответствует рейтингу 3.

mean_rating = median_rating.rating.mean()
max_rating = median_rating.rating.max()
min_rating = median_rating.rating.min()

median_rating['rating_scaled'] = (median_rating.rating - mean_rating) / (max_rating - min_rating)

In [248]:
# получатся значения в диапазоне от -0,01 до 0.32
mean_rating_count = rating_count.userId.mean()
max_rating_count = rating_count.userId.max()
min_rating_count = rating_count.userId.min()


rating_count['rating_count_scaled'] = (rating_count.userId - mean_rating_count)/(max_rating_count - min_rating_count)/3

In [249]:
# получатся значения в диапазоне от -0,004 до 0.49

mean_tag_count = tag_count.tag.mean()
max_tag_count = tag_count.tag.max()
min_tag_count = tag_count.tag.min()

tag_count['tag_count_scaled'] = (tag_count.tag - mean_tag_count) / (max_tag_count - min_tag_count)/2

объединим все рейтинги в один dataframe

In [250]:
joined_movies = movies.join(median_rating.reset_index().set_index('movieId'), on='movieId', lsuffix='_left', rsuffix='_right')



In [251]:
joined_movies = joined_movies.join(rating_count.reset_index().set_index('movieId'), on='movieId', lsuffix='_left', rsuffix='_right')



In [252]:
joined_movies = joined_movies.join(tag_count.reset_index().set_index('movieId'), on='movieId', lsuffix='_left', rsuffix='_right')



In [253]:
joined_movies.head()

,movieId,title,genres,rating,rating_scaled,userId,rating_count_scaled,tag,tag_count_scaled
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,4.0,0.157308,215.0,0.207958,3.0,0.001825
1,2,Jumanji (1995),Adventure|Children|Fantasy,3.5,0.046197,110.0,0.101250,4.0,0.004603
2,3,Grumpier Old Men (1995),Comedy|Romance,3.0,-0.064914,52.0,0.042307,2.0,-0.000952
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance,3.0,-0.064914,7.0,-0.003425,NaN,NaN
4,5,Father of the Bride Part II (1995),Comedy,3.0,-0.064914,49.0,0.039258,2.0,-0.000952


In [269]:
# будем штрафовать, если количество рейтингов ниже 20
def fine_for_low_rating_count(userId):
    if userId <20:
        return -0.2

In [267]:
joined_movies['fine'] = joined_movies.apply(lambda row: fine_for_low_rating_count(row.userId), axis=1)

In [270]:
# заменим отсутствующие данные на 0
joined_movies.fillna(0, inplace = True)

In [280]:
# суммарный score
joined_movies['score'] = joined_movies.rating_count_scaled + joined_movies.rating_scaled + joined_movies.tag_count_scaled + joined_movies.fine

In [281]:
def top_of_genre(df, genre):
    top = df[df.genres.str.contains(genre)].sort_values(by = ['score'], ascending = False)
    return top

In [283]:
# лучшие Comedy

top_of_genre(joined_movies, 'Comedy')[:10]

,movieId,title,genres,rating,rating_scaled,userId,rating_count_scaled,tag,tag_count_scaled,fine,score
257,296,Pulp Fiction (1994),Comedy|Crime|Drama|Thriller,4.50,0.268419,307.0,0.301453,181.0,0.496270,0.0,1.066143
314,356,Forrest Gump (1994),Comedy|Drama|Romance|War,4.00,0.157308,329.0,0.323811,9.0,0.018492,0.0,0.499612
899,1197,"Princess Bride, The (1987)",Action|Adventure|Comedy|Fantasy|Romance,4.50,0.268419,142.0,0.133771,2.0,-0.000952,0.0,0.401238
602,750,Dr. Strangelove or: How I Learned to Stop Worr...,Comedy|War,4.50,0.268419,97.0,0.088039,10.0,0.021270,0.0,0.377728
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,4.00,0.157308,215.0,0.207958,3.0,0.001825,0.0,0.367091
1730,2324,Life Is Beautiful (La Vita è bella) (1997),Comedy|Drama|Romance|War,4.50,0.268419,88.0,0.078892,8.0,0.015714,0.0,0.363026
2996,4011,Snatch (2000),Comedy|Crime|Thriller,4.50,0.268419,93.0,0.083974,0.0,0.000000,0.0,0.352393
863,1136,Monty Python and the Holy Grail (1975),Adventure|Comedy|Fantasy,4.25,0.212864,136.0,0.127673,4.0,0.004603,0.0,0.345140
520,608,Fargo (1996),Comedy|Crime|Drama|Thriller,4.00,0.157308,181.0,0.173405,5.0,0.007381,0.0,0.338094
1298,1732,"Big Lebowski, The (1998)",Comedy|Crime,4.00,0.157308,106.0,0.097185,32.0,0.082381,0.0,0.336874


In [284]:
# лучшие Action

top_of_genre(joined_movies, 'Action')[:10]

,movieId,title,genres,rating,rating_scaled,userId,rating_count_scaled,tag,tag_count_scaled,fine,score
2226,2959,Fight Club (1999),Action|Crime|Drama|Thriller,4.5,0.268419,218.0,0.211006,54.0,0.143492,0.0,0.622918
224,260,Star Wars: Episode IV - A New Hope (1977),Action|Adventure|Sci-Fi,4.5,0.268419,251.0,0.244543,26.0,0.065714,0.0,0.578677
1939,2571,"Matrix, The (1999)",Action|Sci-Fi|Thriller,4.5,0.268419,278.0,0.271982,5.0,0.007381,0.0,0.547782
898,1196,Star Wars: Episode V - The Empire Strikes Back...,Action|Adventure|Sci-Fi,4.5,0.268419,211.0,0.203892,10.0,0.021270,0.0,0.493582
6710,58559,"Dark Knight, The (2008)",Action|Crime|Drama|IMAX,4.5,0.268419,149.0,0.140884,4.0,0.004603,0.0,0.413907
97,110,Braveheart (1995),Action|Drama|War,4.0,0.157308,237.0,0.230315,10.0,0.021270,0.0,0.408893
899,1197,"Princess Bride, The (1987)",Action|Adventure|Comedy|Fantasy|Romance,4.5,0.268419,142.0,0.133771,2.0,-0.000952,0.0,0.401238
507,589,Terminator 2: Judgment Day (1991),Action|Sci-Fi,4.0,0.157308,224.0,0.217104,8.0,0.015714,0.0,0.390126
418,480,Jurassic Park (1993),Action|Adventure|Sci-Fi|Thriller,4.0,0.157308,238.0,0.231331,1.0,-0.003730,0.0,0.384910
254,293,Léon: The Professional (a.k.a. The Professiona...,Action|Crime|Drama|Thriller,4.0,0.157308,133.0,0.124624,35.0,0.090714,0.0,0.372647
